In [2]:
from helpers import *
from Variational_Functions import *

# Run Prediction Evaluations

In [1]:
# -*- coding: utf-8 -*-

unit_list = [31, 24, 25, 30]
unit_dict = {31: 0, 24:1 , 25:2, 30:3}
sample_list = [200, 350, 300, 250, 200]
nm = 125
s_dict = {}
folds = 10
# run through the number of training samples
for ntrain in sample_list:
    u_dict = {}
    for unit in unit_list:
        f_list = []
        for f in range(folds):
            #load the data
            print("sample {}, unit: {}, fold: {}".format(s, u , f))
            Xtot = torch.from_numpy(np.loadtxt('au_data/au_design.txt', delimiter = ",", skiprows = 1)).cuda().double()
            Ytot = torch.from_numpy(pd.read_csv("au_data/au_y.txt").to_numpy()[:, unit_dict[unit]]).cuda().float().double()
            #get cross validation set
            rand_idx = torch.randperm(Xtot.shape[0]).cuda()
            Xtot = Xtot[rand_idx]
            Ytot = Ytot[rand_idx]

            X, Y = Xtot[:ntrain, :], Ytot[:ntrain]
            Xtest, Ytest = Xtot[ntrain:, :], Ytot[ntrain:]
            #normalize the data
            xmean, xs = X.mean(0), X.std(0)
            ymean, ys = Y.mean(0), Y.std(0)
            X = ((X-xmean)/xs)
            Y = ((Y -ymean)/ys)
            Xtest = ((Xtest - xmean)/xs)
            Ytest = ((Ytest - ymean)/ys)
                        
                        
            gpy_dict = gpy_stats(X = X, Y = Y, Xtest = Xtest, Ytest=  Ytest)
            print(gpy_dict)
            #fit all the data
            #Try-Except is for the times there are linear algebra failures in backprop, 
            #1 reinitialization is 
            #almost always sufficient.
            
            try:
                vim2_dict = vim_stats(X = X, Y = Y, Xtest = Xtest, Ytest=  Ytest, 
                                        d1 = 4, J = 2, nm = nm, iters = 2000, 
                                        lr = .0025, nuinit = 1, ft = 1.25, retry = True)
            except:
                vim2_dict = vim_stats(X = X, Y = Y, Xtest = Xtest, Ytest=  Ytest, 
                                        d1 = 4, J = 2 , nm = nm, iters = 2000, lr = .0025,
                                        nuinit = 1,ft = 1.25, 
                                        retry = False,
                                      )

            print(vim2_dict)

            try:
                vim3_dict = vim_stats(X = X, Y = Y, Xtest = Xtest, Ytest=  Ytest, 
                                        d1 = 4, J = 3, nm = nm, 
                                        verbose = False, iters = 4000, lr = .0025, 
                                        nuinit = 3., ft = 1.25)
            except:
                vim3_dict = vim_stats(X = X, Y = Y, Xtest = Xtest, Ytest=  Ytest, 
                                        d1 = 4, J = 3, nm = nm,iters = 2000,
                                        verbose = False, lr = .0025, 
                                        nuinit = 3, ft = 1.25, retry = False)
            try:
                vim4_dict = vim_stats(X = X, Y = Y, Xtest = Xtest, Ytest=  Ytest, 
                                        d1 = 4, J = 4, nm = nm, iters = 2000, 
                                        lr = .0025, nuinit = 1, ft = 1.25)
            except:
                vim4_dict = vim_stats(X = X, Y = Y, Xtest = Xtest, Ytest=  Ytest, 
                                        d1 = 4, J = 4 , nm = nm, iters = 2000, lr = .0025,
                                        nuinit = 1,ft = 1.25, 
                                        retry = False,
                                      )

            
            if vim3_dict['loss'] < vim2_dict['loss']:
                best = 1
            else:
                best = 2
            
            try:
                dle_dict = dle_stats(X = X, Y = Y, Xtest = Xtest, Ytest = Ytest, 
                                     d1 = 5, retry = True)
            except:
                dle_dict = dle_stats(X = X, Y = Y, Xtest = Xtest, Ytest = Ytest, d1 = 5)
            
            try:
                dpa_dict = dpa_stats(X = X, Y = Y, Xtest = Xtest, Ytest = Ytest.float(), 
                                     num_proj = 7, retry = True)
            except:
                dpa_dict = dpa_stats(X = X, Y = Y, Xtest = Xtest, Ytest = Ytest.float(),
                                     num_proj = 7)
            f_dict = {'gpy': gpy_dict, 'vim3': vim3_dict, 'dle': dle_dict, 'dpa': dpa_dict, 'vim2': vim2_dict, 
                      'best': best, 'vim4': vim4_dict}
            print(f_dict)
            f_list.append(f_dict)
        u_dict[unit_dict[unit]] = f_list
    s_dict[s] = u_dict

#save data
import pickle
f = open("au_data/EmuResults.pickle", "wb")
pickle.dump(s_dict, f)
f.close()

sample 200, unit: 31, fold: 0


NameError: name 'torch' is not defined

# Examine Full Data Embedding Matrices

In [ ]:
obs_list = np.genfromtxt('au_data/au_y.txt', dtype=None, delimiter=',', names=True).dtype.names
ntrain = 473
u_list = [24, 25, 30, 31]
unit_dict = {31: 0, 24:1 , 25:2, 30:3}
xcol = np.genfromtxt('au_data/au_design.txt', dtype=None, delimiter=',', names=True).dtype.names
ycol = np.arange(5) + 1
t_list = []
import matplotlib
font = {'family' : 'sans-serif',
        'weight' : 'normal',
        'size'   : 15}

matplotlib.rc('font', **font)

for unit in u_list:
    Xtot = torch.from_numpy(np.loadtxt('au_design.txt', delimiter = ",", skiprows = 1)).cuda().double()
    Ytot = torch.from_numpy(pd.read_csv("au_y.txt").to_numpy()).cuda().float()[:, unit_dict[u]].double()

    rand_idx = torch.randperm(Xtot.shape[0]).cuda()
    Xtot = Xtot[rand_idx]
    Ytot = Ytot[rand_idx]

    X, Y = Xtot[:ntrain, :], Ytot[:ntrain]
    Xtest, Ytest = Xtot[ntrain:(ntrain + 300), :], Ytot[ntrain:(ntrain + 300)]

    xmean, xs = X.mean(0), X.std(0)
    ymean, ys = Y.mean(0), Y.std(0)
    X = ((X-xmean)/xs)
    Y = ((Y -ymean)/ys)
    Xtest = ((Xtest - xmean)/xs)
    Ytest = ((Ytest - ymean)/ys)

    t_list.append(vim_fitb(X = X.double(), Y = Y.double(), d1 = 5,nm =125, nuinit = 1. , tinit = 1., J = 2, lr = .005, iters = 4000, verbose = False))
    

import pickle
f = open("au_data/t_list", "wb")
pickle.dump(t_list, f)
f.close()
    
